In [ ]:
!bash download_voc.sh

--2021-02-21 14:12:41--  http://www.eecs.berkeley.edu/Research/Projects/CS/vision/grouping/semantic_contours/benchmark.tgz
Resolving www.eecs.berkeley.edu (www.eecs.berkeley.edu)... 23.185.0.1, 2620:12a:8000::1, 2620:12a:8001::1
Connecting to www.eecs.berkeley.edu (www.eecs.berkeley.edu)|23.185.0.1|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.eecs.berkeley.edu/Research/Projects/CS/vision/grouping/semantic_contours/benchmark.tgz [following]
--2021-02-21 14:12:41--  https://www.eecs.berkeley.edu/Research/Projects/CS/vision/grouping/semantic_contours/benchmark.tgz
Connecting to www.eecs.berkeley.edu (www.eecs.berkeley.edu)|23.185.0.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www2.eecs.berkeley.edu/Research/Projects/CS/vision/grouping/semantic_contours/benchmark.tgz [following]
--2021-02-21 14:12:41--  https://www2.eecs.berkeley.edu/Research/Projects/CS/vision/grouping/semantic

In [ ]:
!wget 'https://raw.githubusercontent.com/bonlime/keras-deeplab-v3-plus/master/extract_weights.py'
!wget 'https://raw.githubusercontent.com/bonlime/keras-deeplab-v3-plus/master/load_weights.py'
!wget 'https://raw.githubusercontent.com/bonlime/keras-deeplab-v3-plus/master/model.py'

--2021-02-21 14:17:59--  https://raw.githubusercontent.com/bonlime/keras-deeplab-v3-plus/master/extract_weights.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3648 (3.6K) [text/plain]
Saving to: ‘extract_weights.py’

extract_weights.py  100%[===================>]   3.56K  --.-KB/s    in 0s      

2021-02-21 14:18:00 (71.0 MB/s) - ‘extract_weights.py’ saved [3648/3648]

--2021-02-21 14:18:00--  https://raw.githubusercontent.com/bonlime/keras-deeplab-v3-plus/master/load_weights.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 

In [ ]:
from VOCGen import get_voc_datagen
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, LambdaCallback, LearningRateScheduler
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
from model import Deeplabv3
import os
from metrics import mean_iou

In [ ]:
# 20 classes from PASCAL VOC and 1 background
N_CLASS = 20 + 1 
BATCH_SIZE = 12
img_target_size = (320, 320)

In [ ]:
base_lr = 5e-4
max_epoch = 50
power = 0.9

def lr_schedular(epoch, cur_lr):
  if epoch < 5:
    return cur_lr
  return base_lr * ((1 - float(epoch) / max_epoch) ** power)

In [ ]:
img_preproc = lambda x: x / 255.
tr_gen, val_gen = get_voc_datagen(N_CLASS, BATCH_SIZE, img_preproc, None, img_target_size)

In [ ]:
model = Deeplabv3(input_shape=(img_target_size[0], img_target_size[1], 3), backbone='xception', activation='softmax')

166100992/166096424 [==============================] - 2s 0us/step


In [ ]:
opt = Adam(base_lr)
model_save_dir = os.path.join('drive', 'MyDrive', 'adverseg_models', 'deeplabv3_xception_320.h5')
checkpoint = ModelCheckpoint(model_save_dir, verbose=True, save_best_only=True, monitor='val_mean_iou', mode='max')
scheduleLR = LearningRateScheduler(lr_schedular, verbose=True)
model.compile(opt, 'categorical_crossentropy', metrics=['accuracy', mean_iou])

In [ ]:
model.fit(tr_gen, epochs=50, validation_data=val_gen, callbacks=[checkpoint, scheduleLR])

Epoch 1/50

Epoch 00001: LearningRateScheduler reducing learning rate to 0.0005000000237487257.
709/709 [==============================] - 861s 1s/step - loss: 0.6421 - accuracy: 0.8156 - mean_iou: 0.4418 - val_loss: 0.6697 - val_accuracy: 0.8281 - val_mean_iou: 0.5057

Epoch 00001: val_mean_iou improved from -inf to 0.50567, saving model to drive/MyDrive/adverseg_models/deeplabv3_xception_320.h5
Epoch 2/50

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0005000000237487257.
709/709 [==============================] - 858s 1s/step - loss: 0.4567 - accuracy: 0.8602 - mean_iou: 0.5197 - val_loss: 0.4974 - val_accuracy: 0.8489 - val_mean_iou: 0.5588

Epoch 00002: val_mean_iou improved from 0.50567 to 0.55879, saving model to drive/MyDrive/adverseg_models/deeplabv3_xception_320.h5
Epoch 3/50

Epoch 00003: LearningRateScheduler reducing learning rate to 0.0005000000237487257.
709/709 [==============================] - 860s 1s/step - loss: 0.3888 - accuracy: 0.8785 - mean_iou: